In [50]:
import pandas as pd
import requests
import time


In [51]:
df = pd.read_csv("cross-species.csv")
df = df[["Protein_ID", "meltPoint"]]
df["Uniprot"] = df["Protein_ID"].str.split("_").str[0]
df = df.dropna()
df

/var/folders/83/05d_l9fn2q3916cmlh6jsskw0000gq/T/ipykernel_29186/2891718019.py:1: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cross-species.csv")


,Protein_ID,meltPoint,Uniprot
1,C0H3V2_mtlF,56.693844,C0H3V2
2,C0H3V8_yyzM,43.373369,C0H3V8
3,C0H3Y1_yhzD,40.693719,C0H3Y1
5,C0H405_ykzS,48.069618,C0H405
7,C0H437_yoyH,39.507236,C0H437
...,...,...,...
456248,R4YVD3_gpm,45.057271,R4YVD3
456249,R4YVD4_prlC,40.407357,R4YVD4
456250,R4YVD6_OLEAN_C38380,30.038882,R4YVD6
456253,R4YVD9_argB,40.143480,R4YVD9


In [ ]:

def fetch_sequences_batch(ids, batch_size=100, sleep=1.0):
    """
    Fetch UniProt sequences in batches using the UniProt API.
    Returns dict {uniprot_id: sequence}.
    """
    seqs = {}
    for i in range(0, len(ids), batch_size):
        batch = ids[i:i+batch_size]
        query = " OR ".join([f"accession:{uid}" for uid in batch])

        url = "https://rest.uniprot.org/uniprotkb/stream"
        params = {
            "format": "fasta",
            "query": query
        }

        response = requests.get(url, params=params)

        if response.status_code == 200:
            fasta = response.text.strip().split("\n>")
            for entry in fasta:
                lines = entry.strip().split("\n")
                if not lines or not lines[0].strip():
                    continue  # skip empty entries

                header = lines[0].replace(">", "").strip()
                seq = "".join(lines[1:]).strip()

                parts = header.split("|")
                if len(parts) >= 2:
                    acc = parts[1]
                else:
                    tokens = header.split()
                    if tokens:   # check non-empty
                        acc = tokens[0]
                    else:
                        continue  # skip if completely malformed
                seqs[acc] = seq
        else:
            print(f"Batch {i//batch_size+1}: Error {response.status_code} — {response.text}")

        time.sleep(sleep)  # avoid hammering the API
    return seqs

# Get unique IDs and fetch sequences
unique_ids = df["Uniprot"].unique().tolist()
seq_dict = fetch_sequences_batch(unique_ids, batch_size=100)

# Map sequences back into DataFrame
df["Sequence"] = df["Uniprot"].map(seq_dict)

df.to_csv("meltome_cleaned.csv", index = False)

In [ ]:
df = df.dropna()
df = df[["meltPoint", "Sequence"]]
df.to_csv("meltome.csv", index = False)
